In [1]:
pip install pinecone


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/student/.virtualenvs/final-project/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import json
import zipfile
from pathlib import Path
import pdfplumber
import docx
from pdfminer.high_level import extract_text
import nltk
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from tqdm import tqdm
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    SpacyTextSplitter
)
from pinecone import Pinecone, ServerlessSpec
import spacy
import subprocess
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

In [3]:
def ensure_spacy_model(model_name="en_core_web_sm"):
    """Ensure that the given spaCy model is installed."""
    try:
        spacy.load(model_name)
    except OSError:
        print(f"⚠️ spaCy model '{model_name}' not found. Downloading...")
        subprocess.run(["python", "-m", "spacy", "download", model_name], check=True)
        print(f"✅ Downloaded spaCy model: {model_name}")

In [4]:
def recursive_chunking(texts: list[str], chunk_size: int, overlap: int) -> list[str]:
    char_size = chunk_size * AVG_WORD_LEN
    char_overlap = overlap * AVG_WORD_LEN
    splitter = RecursiveCharacterTextSplitter(chunk_size=char_size, chunk_overlap=char_overlap)
    chunks = []
    for text in texts:
        chunks.extend(splitter.split_text(text))
    return chunks

In [32]:
def overlapping_chunking(texts: list[str], chunk_size: int, overlap: int) -> list[str]:
    chunks = []
    step = chunk_size - overlap
    for text in texts:
        words = text.split()
        for i in range(0, len(words), step):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk:
                chunks.append(chunk.strip())
    return chunks

In [31]:
def spacy_chunking(texts: list[str], chunk_size: int, overlap: int) -> list[str]:
    char_size = chunk_size * AVG_WORD_LEN
    char_overlap = overlap * AVG_WORD_LEN
    splitter = SpacyTextSplitter(chunk_size=char_size, chunk_overlap=char_overlap)
    chunks = []
    for text in texts:
        chunks.extend(splitter.split_text(text))
    return chunks

In [7]:
def select_chunking_strategy(strategy_name: str):
    strategies = {
        "recursive": recursive_chunking,
        "overlapping": overlapping_chunking,
        "spacy": spacy_chunking,
    }
    return strategies.get(strategy_name)

In [8]:
def extract_zip_file(zip_path: Path, extract_to: Path):
    if not extract_to.exists():
        extract_to.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_path} to {extract_to}")

def remove_cid_artifacts(text):
    return re.sub(r'\(cid:\d+\)', '', text)

def read_pdf(path: Path) -> str:
    try:
        text = extract_text(str(path))
    except Exception as e:
        print(f"⚠️ Error reading {path.name}: {e}")
        text = ""
    return remove_cid_artifacts(text)

def read_docx(path: Path) -> str:
    doc = docx.Document(str(path))
    return "\n".join(p.text for p in doc.paragraphs)

def clean_text(text: str) -> str:
    text = re.sub(r"[^a-zA-Z0-9\.\,\;\:\?\!\-\s]", " ", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n+", "\n", text)
    return text.strip()

def embed_chunks(texts: list[str], model: SentenceTransformer) -> np.ndarray:
    if not texts:
        return np.empty((0, model.get_sentence_embedding_dimension()), dtype="float32")
    embs = model.encode(texts, convert_to_numpy=True)
    return normalize(embs, axis=1).astype("float32")

In [9]:
def create_index(pc, index_name, dimension, metric):
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric = metric,
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print(f"Created new Pinecone index: {index_name}")
    else:
        print(f"Using existing Pinecone index: {index_name}")

def save_to_pinecone(records, embeddings, pc, index_name, namespace="default"):
    index = pc.Index(index_name)
    vectors = []
    for i, record in enumerate(records):
        vectors.append({
            "id": f"{Path(record['file']).stem}_chunk_{record['chunk_id']}",
            "values": embeddings[i].tolist(),
            "metadata": {
                "file": Path(record['file']).name,
                "chunk_id": record['chunk_id'],
                "text": record['text'],
                "length": record['length'],
            }
        })
    for i in range(0, len(vectors), 100):
        index.upsert(vectors=vectors[i:i+100], namespace=namespace)
    #print(f"📌 Saved {len(vectors)} vectors to Pinecone namespace '{namespace}'.")

In [36]:
MODEL_NAME = "multi-qa-mpnet-base-dot-v1"
ZIP_PATH = Path("dataset.zip")
DATA_DIR = ZIP_PATH.with_suffix("")
WORDS_PER_CHUNK = 700
WORDS_OVERLAP = 200
AVG_WORD_LEN = 6
PINECONE_INDEX_NAME = f'dotproduct-{WORDS_PER_CHUNK}'
CHUNK_STRATEGIES = ["spacy", "recursive", "overlapping"]  # Options: recursive / overlapping / spacy

with open(r"src/api_keys.json") as f:
    api_keys = json.load(f)
pc = Pinecone(api_key=api_keys["pinecone"])

In [ ]:
# Loading model
print(f"[1/3] Loading model '{MODEL_NAME}'...")
model = SentenceTransformer(MODEL_NAME)
dimension = model.get_sentence_embedding_dimension()
metric = PINECONE_INDEX_NAME.split("-")[0]
create_index(pc, PINECONE_INDEX_NAME, dimension, metric)

In [ ]:
print("[2/3] Extracting zip and reading documents...")
extract_zip_file(ZIP_PATH, DATA_DIR)
raw_texts = []
sources = []
all_files = list(DATA_DIR.rglob("*"))
with tqdm(all_files, desc="Reading files") as pbar:
    for path in pbar:
        if path.suffix.lower() == ".pdf":
            text = read_pdf(path)
        elif path.suffix.lower() == ".docx":
            text = read_docx(path)
        else:
            continue
        cleaned = clean_text(text)
        raw_texts.append(cleaned)
        sources.append(str(path))

In [ ]:
print("[3/3] Chunking, embedding and uploading text...")
for strategy in CHUNK_STRATEGIES:
    if strategy == "spacy":
      ensure_spacy_model("en_core_web_sm")

    chunk_func = select_chunking_strategy(strategy)
    records = []
    for file_path, text in zip(sources, raw_texts):
        file_chunks = chunk_func([text], chunk_size=WORDS_PER_CHUNK, overlap=WORDS_OVERLAP)
        file_name = Path(file_path).stem
        for i, chunk in enumerate(file_chunks):
            records.append({
                "file": file_name,
                "chunk_id": i,
                "text": chunk,
                "length": len(chunk.split()),
            })

    print(f"{strategy} → Created {len(records)} chunks")
    embs = embed_chunks([r["text"] for r in records], model)
    namespace = strategy
    save_to_pinecone(records, embs, pc, PINECONE_INDEX_NAME, namespace=namespace)
    #print(f"{strategy} → Uploaded {len(records)} chunks to Pinecone namespace '{namespace}'.")
